In [1]:
import spotipy
import spotipy.util as util
import os
from json import load

# make the notebook automatically reload external python modules
%load_ext autoreload
%autoreload 2

In [2]:
with open('credentials.txt') as fd:
    creds = load(fd)

In [3]:
# For the purposes of this notebook we only want to read from a users library
scope = 'user-library-read, playlist-modify-public, playlist-modify-private'

In [4]:
def get_token(scope, **kwargs):
    try:
        token = util.prompt_for_user_token(**kwargs, scope=scope)
    except:
        os.remove(f".cache-{kwargs['username']}")
        token = util.prompt_for_user_token(**kwargs, scope=scope)
    return token

In [5]:
token = get_token(scope, **creds)

In [6]:
sp = spotipy.Spotify(auth=token)

In [7]:
# some of the methods require your user id so its a good idea to save it now 
user = sp.current_user()
user_id = user['id']

In [8]:
def get_playlist_track_names_ids(playlist_id):
    """
    This function returns two lists where the first is the names
    of every track and the second are the associated track ids. 
    """
    global user_id
    playlist_info = sp.user_playlist(user_id, playlist_id=playlist_id)
    tracks = playlist_info['tracks']['items']
    names, ids = [], []
    
    for track in tracks:
        names.append(track['track']['name'])
        ids.append(track['track']['id'])
    
    return names, ids

In [9]:
def get_album_track_names_ids(album_id):
    """
    This function returns two lists where the first has the names 
    of every track and the second are the associated track ids. 
    """
    album = sp.album_tracks(album_id)
    names, ids = [], []
    
    for track in album['items']:
        names.append(track['name'])
        ids.append(track['id'])
    
    return names, ids

In [10]:
# Get all the track names and ids from the Chill Hits playlist
names, ids = get_playlist_track_names_ids('spotify:playlist:37i9dQZF1DX4WYpdgoIcn6')

## Reading all of a users playlists

In [11]:
my_playlists = sp.current_user_playlists()

In [12]:
my_playlists['items'][0]['name']

'For The Throne - Music Inspired by the HBO Series Game of Thrones'

In [13]:
for pl in my_playlists['items']:
    print(pl['name'])

For The Throne - Music Inspired by the HBO Series Game of Thrones
Osher showed me this
Chill Hits
Tastebreakers
Jazztronica
My Shazam Tracks
Background
Chill Vibes
Rena's bomb DJ skils 
Jazz Vibes
Peaceful Guitar
Soul.House.Relax
State of Jazz
take a trip
הכל סבבה
Liked from Radio
Avenged Sevenfold Discography
Discover Weekly
Fresh Finds
Reggae/Hazy Dayz (#feelingGreen)
Coffee Table Jazz
Intense Studying
Feel-Good Indie Rock
Rock Ballads
Sax and the City
מלך
Muki
Travis Scott
Calvin Harris
Kid Cudi
Unlike Pluto
JAZZ
Various Song Downloads
Avenged Sevenfold
John Mayer


## Creating a Playlist

In this section well create our own playlist which will be a clone of the Chill Hits playlist.

When you create a playlist the method `sp.user_playlist_create` returns a dictionary containing the information of the newly created playlist including its name and id. We can take the id to the add songs to the playlist and delete songs from it.

In [14]:
# First, we create the playlist. It should appear in your playlists now in spotify.
pl = sp.user_playlist_create(user_id, 'Chill Hits Clone')
pl

{'collaborative': False,
 'description': None,
 'external_urls': {'spotify': 'https://open.spotify.com/playlist/63PSKVCuLUyObFfvXfRBy3'},
 'followers': {'href': None, 'total': 0},
 'href': 'https://api.spotify.com/v1/playlists/63PSKVCuLUyObFfvXfRBy3',
 'id': '63PSKVCuLUyObFfvXfRBy3',
 'images': [],
 'name': 'Chill Hits Clone',
 'owner': {'display_name': 'eitanohana',
  'external_urls': {'spotify': 'https://open.spotify.com/user/eitanohana'},
  'href': 'https://api.spotify.com/v1/users/eitanohana',
  'id': 'eitanohana',
  'type': 'user',
  'uri': 'spotify:user:eitanohana'},
 'primary_color': None,
 'public': True,
 'snapshot_id': 'MSxkOWU5YjI4ZGQyMzFjZWU4NzExZmVkZmVmZWU2NDc1NmQ2ZDAwZmRk',
 'tracks': {'href': 'https://api.spotify.com/v1/playlists/63PSKVCuLUyObFfvXfRBy3/tracks',
  'items': [],
  'limit': 100,
  'next': None,
  'offset': 0,
  'previous': None,
  'total': 0},
 'type': 'playlist',
 'uri': 'spotify:playlist:63PSKVCuLUyObFfvXfRBy3'}

In [15]:
pl['id']

'63PSKVCuLUyObFfvXfRBy3'

In [16]:
# And we need to get the ids from the Chill Hits playlist
_, track_ids = get_playlist_track_names_ids('spotify:playlist:37i9dQZF1DX4WYpdgoIcn6')

In [17]:
# Now we add all the tracks to the newly created playlist
sp.user_playlist_add_tracks(user_id, pl['id'], track_ids)

{'snapshot_id': 'MiwzZjViYzg1Y2RjMzM0ZDhlMzU1YzljNTRjN2ZhNjlmZTMwNjU2YTEw'}